In [ ]:

import pandas as pd

from utils.fundamentals import get_fundamentals
from utils.industry import get_industry_classification
from utils.pandas_utils import merge_df_safely
from utils.rating import get_ratings
from utils.technical import get_technicals
from utils.tradingview import TradingView
from utils.pandas_utils import make_df_ready_for_serialization
from utils.bucket import storage_options

In [ ]:
df = TradingView.get_base_symbols(1000)

In [ ]:
from provider.tradingview.quote_scaler import TradingViewScaler, QuoteStreamEvent

fields = [
    "ch",
    "chp",
    "current_session",
    "lp",
    "lp_time",
    "pro_name",
    "short_name",
    "type",
    "update_mode",
    "volume",
]
scaler = TradingViewScaler(quote_fields=fields)

In [ ]:
scaler.start()

In [ ]:
await scaler.add_tickers(df.index.to_list())

In [ ]:
async for event_type, ticker, data in scaler.quote_events():
    if event_type == QuoteStreamEvent.QUOTE_UPDATE:
            pass